# GreenValueNet

This notebook contains the code needed to execute the GreenValueNet hedonic pricing neural network. 

In [32]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
from data_load_funcs import get_params, load_data_catalogue
from processing_funcs import process_data, normalise_values

cwd = Path.cwd()
params = get_params()
data_catalogue = load_data_catalogue()
print(params)

# check inputs func
# checks for processed file
# if not creates at highest level
# if not those goes back
# does again until raw files are checked

# in process_data, either load it in, or create it
# in process_housing_data, either load it in, or create it
# same in spatial data


{'chunksize': 100000, 'spatial_dict': {'coastline': {'folder': 'processed_inputs', 'file': 'coastline.shp'}, 'prim_school': {'folder': 'processed_inputs', 'file': 'primary_school.shp'}, 'sec_school': {'folder': 'processed_inputs', 'file': 'secondary_school.shp'}, 'roads': {'folder': 'processed_inputs', 'file': 'roads_c.shp'}, 'nat_park': {'folder': 'raw_inputs', 'file': 'National_Parks_(England)___Natural_England.shp'}, 'nat_trust': {'folder': 'raw_inputs', 'file': 'NT_Land_Always_Open.shp'}, 'rail_station': {'folder': 'raw_inputs', 'file': 'Railway_Stations.shp'}, 'ttwa': {'folder': 'raw_inputs', 'file': 'TTWA2011.shp'}}, 'pre_processing': {'postcodes': {'file_name': 'postcodes_c.shp', 'func': 'concat_postcodes'}, 'roads': {'file_name': 'roads_c.shp', 'func': 'concat_roads'}, 'coastline': {'file_name': 'coastline.shp', 'func': 'make_coastline'}, 'mapped_postcodes': {'processed_file': 'mapped_postcodes.shp'}}, 'house_prices': {'processed_file': 'house_prices_c.csv'}, 'spatial_attribute

In [33]:
dataset = process_data(data_catalogue, params)

# print summary stats table here
# extract norm cols from params
# for col in norm_cols:
#     dataset[col] = normalise_values(dataset[col])

TypeError: unsupported operand type(s) for /: 'WindowsPath' and 'dict'